In [126]:
import pandas as pd
from pathlib import Path
import os
import datetime
import holidays
import numpy as np
import calendar

# Control pre-processing 
 - define whether to drop rows (only after expl. data analysis)

In [127]:
drop_graz_sahara_measurements = True
drop_graz_newYear_measurements = True

drop_zagreb_sahara_measurements = True
drop_zagreb_newYear_measurements = True

add_lag_values = True

dayOfYearSinusFunction = False

### define functions used for adding features in both data sets

In [128]:
# the below is the same, but takes a string:
at_holidays = holidays.country_holidays(subdiv='6',country="AT")
hr_holidays = holidays.country_holidays(country="HR")
def check_holiday(date,country_hol)->int:
    if date in country_hol:
        return 1
    else:
        return 0
        
def check_weekend(date)->int:
    if date.weekday() > 4: #goes from 0..6
        return 1
    else:
        return 0

def get_day_of_year(date):
    return date.timetuple().tm_yday


# Function to get the day of the year dynamically considering leap years
def get_day_of_year_adjusted(date):
    year = date.year
    days_in_year = 366 if calendar.isleap(year) else 365
    return date.timetuple().tm_yday, days_in_year

# Pre-process data from Zagreb
 - read 
 - rename variables (columns) -> done in excel sheet
 - replace/write C with/in ene
 - delete NEE since it is always 0 -> after that there are 16 categories (which is the norm)
 - save as csv

In [129]:
def read_excel_file(path,col_names=None) -> pd.DataFrame:
    data_dict = pd.read_excel(path,
                sheet_name=None,
                index_col=0,
                header=0,
                names=col_names,
                parse_dates=[0])
    return data_dict[list(data_dict.keys())[0]]

filepath = Path('../datasets/data_zagreb.xlsx')

col_names_zagreb = ['z_pm10', 'z_pm2.5', 'z_pm1', 'year', 'z_tempMax', 'z_tempMin', 'z_temp',
                   'z_tempMax-min', 'z_pressureMax', 'z_pressureMin', 'z_pressure', 'z_pressurePmax-min', 'z_rhMax',
                   'z_rh_min', 'z_rh', 'z_rh_max-min', 'z_windsp', 'z_precip',
                   "n","nne","ne","ene","e","ese", "se", "sse","s","ssw","sw","wsw","w","wnw","nw","nnw",'nee','c']


df_zagreb = read_excel_file(filepath,col_names_zagreb)

## replace/write C with/in ene AND delete col c
## replace nee with ene (there is only one nee value: 2013-03-14) 

In [ ]:
df_zagreb.loc[df_zagreb['c'] == 1, 'ene'] = 1
df_zagreb.drop('c', inplace=True, axis=1)

df_zagreb.loc[df_zagreb['nee'] == 1, 'ene'] = 1
df_zagreb.drop('nee', inplace=True, axis=1)

df_zagreb

In [131]:
def create_wind_dir_list(df) -> list:
    """
    creates a list of wind directions e.g. 1->e, 2->ene,...
    """
    # in this order the columns occur in the sheet
    li_wind_key = ["n","nne","ne","ene","e","ese", "se", "sse","s","ssw","sw","wsw","w","wnw","nw","nnw"]

    li_wind_val = list(range(1,17))
    dict_wind_dir = dict(zip(li_wind_key, li_wind_val))
    li_wind_dir_category = []
    for index, row in df.iterrows():
        found = False
        for key,value in dict_wind_dir.items():
            if row[key] == 1:
                li_wind_dir_category.append(value)
                found=True
        if found==False:
            print(row)
    return li_wind_dir_category

def wind_class_to_degree(w_class_number) -> float:
    li_wind_key = ["n","nne","ne","ene","e","ese", "se", "sse","s","ssw","sw","wsw","w","wnw","nw","nnw"]
    w_class = li_wind_key[w_class_number-1]
    dict_degs = {'n': 0,
            'nne': 22.5,
            'ne': 45,
            'ene': 67.5,
            'e': 90,
            'ese': 112.5,
            'se': 135,
            'sse': 157.5,
            's': 180,
            'ssw': 202.5,
            'sw': 225,
            'wsw': 247.5,
            'w': 270,
            'wnw': 292.5,
            'nw': 315,
            'nnw': 337.5}
    return dict_degs[w_class]

li_calc_class = create_wind_dir_list(df_zagreb)
li_calc_degree = [wind_class_to_degree(elem) for elem in li_calc_class]
# add new column and fill with values
df_zagreb.insert(17, "z_windDirClass",li_calc_class)
df_zagreb.insert(18, "z_windDirDeg",li_calc_degree)


# Add features:
 - day of year
 - isholiday
 - dayBeforH
 - dayAfterH

In [132]:

# OLD without sin transformation
if dayOfYearSinusFunction:
    df_zagreb["dayOfYear"] = [
    np.sin(2 * np.pi * get_day_of_year_adjusted(date)[0] / get_day_of_year_adjusted(date)[1])
    for date in df_zagreb.index.to_pydatetime()
]
else:
    df_zagreb["dayOfYear"] =  [get_day_of_year(date) for date in df_zagreb.index.to_pydatetime()]


df_zagreb["holiday"] =  [check_holiday(date,hr_holidays) for date in df_zagreb.index]
df_zagreb["dayAfterHoliday"] =  [check_holiday(date-datetime.timedelta(days=1),hr_holidays) for date in df_zagreb.index]
df_zagreb["dayBeforeHoliday"] =  [check_holiday(date+datetime.timedelta(days=1),hr_holidays) for date in df_zagreb.index]
df_zagreb["weekend"] =  [check_weekend(date) for date in df_zagreb.index]
if add_lag_values:
    df_zagreb["pm10Lag"] = df_zagreb["z_pm10"].shift(1) 



## Define a function to output nan values

In [ ]:
def calc_output_nans(df):
    total_nan = 0
    for elem in list(df.keys()):
        length = len(df.loc[df[elem].isnull(),])
        total_nan += length
        if length > 0:
            print("Feature: "+ elem, length)
    print("Total NaN: ",total_nan)

calc_output_nans(df_zagreb)

# Drop data from 24th to 28th March (Sahara Dust)

In [ ]:

print("Sahara dust values")
display(df_zagreb[(df_zagreb.index >= "2020-03-24") & (df_zagreb.index <= "2020-03-28")])

index_to_drop_zagreb = df_zagreb[(df_zagreb.index >= "2020-03-24") & (df_zagreb.index <= "2020-03-28")].index

if drop_zagreb_sahara_measurements:
    print("******************************\nDrop Sahara measurements\n******************************")
    df_zagreb.drop(index_to_drop_zagreb,inplace=True)


# Drop data annually 1st to 3rd January (New year)

In [ ]:
li_drop_dates_new_year = ['01-01', '01-02','01-03']

print("New year values")
display(df_zagreb[df_zagreb.index.strftime('%m-%d').isin(li_drop_dates_new_year)])

index_to_drop_zagreb = df_zagreb[df_zagreb.index.strftime('%m-%d').isin(li_drop_dates_new_year)].index

if drop_zagreb_newYear_measurements:
    print("******************************\nDrop NewYear measurements\n******************************")
    df_zagreb.drop(index_to_drop_zagreb,inplace=True)
    

# Save pre-processed data from Zagreb

In [136]:
filepath = Path('../datasets/data_zagreb_preprocessed.csv')  

# uncomment to save
#df_zagreb.to_csv(filepath,sep=',')  

# Select desired features

In [ ]:
if add_lag_values:
    df_zagreb = df_zagreb[['z_pm10', 'z_pm2.5', 'z_pm1', 'year',
           'z_temp','z_pressure', 'z_rh','z_windsp', 'z_windDirClass', 'z_windDirDeg',
           'z_precip', 'dayOfYear', 'holiday','dayAfterHoliday', 'dayBeforeHoliday', 'weekend',"pm10Lag"]]
else:
     df_zagreb = df_zagreb[['z_pm10', 'z_pm2.5', 'z_pm1', 'year',
           'z_temp','z_pressure', 'z_rh','z_windsp', 'z_windDirClass', 'z_windDirDeg',
           'z_precip', 'dayOfYear', 'holiday','dayAfterHoliday', 'dayBeforeHoliday', 'weekend']]
    
df_zagreb.keys()

# Rename columns

In [138]:
def rename_columns(df)->pd.DataFrame:
    li_col_names = df.columns
    li_new_names = [elem.split('_')[1] if len(elem.split('_'))>1 else elem for elem in li_col_names]
    
    dict_rename = dict(zip(li_col_names,li_new_names))
    return  df.rename(columns=dict_rename)

df_zagreb = rename_columns(df_zagreb)

# Save pre-processed data from Zagreb as done per Station in Graz

In [139]:
filepath_zagreb = Path('../datasets/data_per_station/z.csv')

# uncomment to save
df_zagreb.to_csv(filepath_zagreb,sep=',') 


# Pre-process data from Graz
 - read 
 - rename variables (columns) -> done in excel sheet
 - replace/write C with/in ene
 - delete NEE since it is always 0 -> after that there are 16 categories (which is the norm)
 - save as csv

In [140]:
filepath_graz_met = Path('../datasets/data_graz_meteorology.csv')
filepath_graz_pol = Path('../datasets/data_graz_air_pollutants.csv')

def read_csv_data(path,col_names=None) -> pd.DataFrame:
    ts = pd.read_csv(path,
        header=0,
        names=col_names,
        index_col=0,
        infer_datetime_format=True,
        parse_dates=[0]) #to set index to datetime
    return ts

In [141]:
col_names_met = ['d_temp', 'd_rh', 'n_temp', 'n_rh', 'n_pressure',
       'n_precip', 'n_radiation', 'n_windDirDeg', 'n_windsp',
       'n_windPeak', 'e_temp', 'e_rh', 'e_pressure',
       'e_windDirDeg', 'e_windsp', 'e_windPeak', 's_temp',
       's_rh', 's_windDirDeg', 's_windsp', 's_windPeak',
       'w_temp', 'w_rh', 'w_windDirDeg', 'w_windsp',
       'w_windPeak']
col_names_pol = ['d_no', 'd_no2', 'd_nox', 'd_pm10', 'n_o3', 'n_no', 'n_no2', 'n_nox',
       'n_pm10', 'e_no', 'e_no2', 'e_nox', 'e_pm10', 's_o3', 's_no', 's_no2',
       's_nox', 's_pm10', 'w_no', 'w_no2', 'w_nox', 'w_pm10']
df_graz_met = read_csv_data(filepath_graz_met,col_names_met)
df_graz_pol = read_csv_data(filepath_graz_pol,col_names_pol)

calc wind direction degrees into classes:
https://www.surfertoday.com/windsurfing/how-to-read-wind-direction

In [142]:
def degToCategorical(num,li_str):
    index=int((int(num)/22.5)+0.5)
    li = list(range(0,17))
    if li_str:
        li=["n","nne","ne","ene","e","ese", "se", "sse","s","ssw","sw","wsw","w","wnw","nw","nnw"]
    return li[(index % 16)]

In [ ]:
df_graz_met.keys()

In [ ]:

print(degToCategorical(348,True))
#int(154.7/22.5)

### Add column wind directions classes, which are calculated of wind direction

In [145]:
import math

for winddirdeg,winddirclass in [('n_windDirDeg','n_windDirClass'),('e_windDirDeg','e_windDirClass'),('s_windDirDeg','s_windDirClass'),('w_windDirDeg','w_windDirClass')]:
    df_graz_met[winddirclass] = df_graz_met[winddirdeg].apply(lambda x: degToCategorical(x,False) if not math.isnan(x) else x)

# first change fill nan values with -99 to cast column into int -> not possible with NaN
df_graz_met = df_graz_met.fillna(-99)

for elem in ['n_windDirClass','e_windDirClass','s_windDirClass','w_windDirClass']:
       df_graz_met[elem] = df_graz_met[elem].astype(int)
 

df_graz_met.replace(-99.000000,np.nan,inplace=True)


## Define a function to output nan values

In [ ]:
def calc_output_nans(df):
    total_nan = 0
    for elem in list(df.keys()):
        length = len(df.loc[df[elem].isnull(),])
        total_nan += length
        if length > 0:
            print("Feature: "+ elem, length)
    print("Total NaN: ",total_nan)

calc_output_nans(df_graz_pol)

## Impute empty temperature,pressure,humidity values with the mean temperature of the remaining stations: 
 - calc if depending on the values, if none value do not count that stations. sum/n -> n depends on how many none NaN values exist

In [ ]:
calc_output_nans(df_graz_met)

In [148]:
def calc_mean(row):
    li = row
    li_calc = []
    cnt = 0
    for entry in row:
        #print(elem)
        if not math.isnan(entry):
            cnt += 1
            li_calc.append(entry)
    #print(sum(li_calc),cnt)
    return sum(li_calc)/cnt

# only set a value if value is NaN
def set_value_if_none(row):
    if math.isnan(row[1]):
        return row[0]
    else:
        return row[1]

#apparently temp of donbosco is not a float, needs to be casted
df_graz_met["d_temp"] = pd.to_numeric(df_graz_met["d_temp"])

# MEAN VALUE IMPUTATION OLD
# --------------------------------------------------------------------------------------------------------------
# # calc mean temp and set
# df_graz_met["mean_temp"] = df_graz_met[["d_temp","n_temp","s_temp","w_temp","e_temp"]].apply(calc_mean,axis=1)
# df_graz_met['e_temp'] = df_graz_met[["mean_temp","e_temp"]].apply(set_value_if_none,axis=1)
# df_graz_met['n_temp'] = df_graz_met[["mean_temp","n_temp"]].apply(set_value_if_none,axis=1)
# df_graz_met['s_temp'] = df_graz_met[["mean_temp","s_temp"]].apply(set_value_if_none,axis=1)
# df_graz_met['w_temp'] = df_graz_met[["mean_temp","w_temp"]].apply(set_value_if_none,axis=1)
# df_graz_met['d_temp'] = df_graz_met[["mean_temp","d_temp"]].apply(set_value_if_none,axis=1)

# df_graz_met.drop(columns=["mean_temp"],inplace=True)

# # set pressure, only 2 stations with feature pressure, mean cannot be calculated if one is missing
# df_graz_met['e_pressure'] = df_graz_met[["n_pressure","e_pressure"]].apply(set_value_if_none,axis=1)
# df_graz_met['n_pressure'] = df_graz_met[["e_pressure","n_pressure"]].apply(set_value_if_none,axis=1)


# #calc mean humidity and set
# df_graz_met["mean_rh"] = df_graz_met[["d_rh","n_rh","e_rh","s_rh","w_rh"]].apply(calc_mean,axis=1)
# df_graz_met['e_rh'] = df_graz_met[["mean_rh","e_rh"]].apply(set_value_if_none,axis=1)
# df_graz_met['d_rh'] = df_graz_met[["mean_rh","d_rh"]].apply(set_value_if_none,axis=1)
# df_graz_met['n_rh'] = df_graz_met[["mean_rh","n_rh"]].apply(set_value_if_none,axis=1)
# df_graz_met['s_rh'] = df_graz_met[["mean_rh","s_rh"]].apply(set_value_if_none,axis=1)
# df_graz_met['w_rh'] = df_graz_met[["mean_rh","w_rh"]].apply(set_value_if_none,axis=1)

# df_graz_met.drop(columns=["mean_rh"],inplace=True)

# --------------------------------------------------------------------------------------------------------------

# VALUE IMPUTATION FROM NEARBY STATION NEW 


df_graz_met['e_temp'] = df_graz_met[["s_temp","e_temp"]].apply(set_value_if_none,axis=1)
df_graz_met['n_temp'] = df_graz_met[["w_temp","n_temp"]].apply(set_value_if_none,axis=1)
df_graz_met['s_temp'] = df_graz_met[["e_temp","s_temp"]].apply(set_value_if_none,axis=1)
df_graz_met['w_temp'] = df_graz_met[["d_temp","w_temp"]].apply(set_value_if_none,axis=1)
df_graz_met['d_temp'] = df_graz_met[["w_temp","d_temp"]].apply(set_value_if_none,axis=1)

df_graz_met['e_pressure'] = df_graz_met[["n_pressure","e_pressure"]].apply(set_value_if_none,axis=1)
df_graz_met['n_pressure'] = df_graz_met[["e_pressure","n_pressure"]].apply(set_value_if_none,axis=1)

df_graz_met['e_rh'] = df_graz_met[["s_rh","e_rh"]].apply(set_value_if_none,axis=1)
df_graz_met['d_rh'] = df_graz_met[["w_rh","d_rh"]].apply(set_value_if_none,axis=1)
df_graz_met['n_rh'] = df_graz_met[["w_rh","n_rh"]].apply(set_value_if_none,axis=1)
df_graz_met['s_rh'] = df_graz_met[["e_rh","s_rh"]].apply(set_value_if_none,axis=1)
df_graz_met['w_rh'] = df_graz_met[["d_rh","w_rh"]].apply(set_value_if_none,axis=1)

In [ ]:
calc_output_nans(df_graz_met)

# ADD PM10 LAG Values

In [150]:
if add_lag_values:
    df_graz_pol["d_pm10Lag"] = df_graz_pol["d_pm10"].shift(1)
    df_graz_pol["n_pm10Lag"] = df_graz_pol["n_pm10"].shift(1) 
    df_graz_pol["e_pm10Lag"] = df_graz_pol["e_pm10"].shift(1) 
    df_graz_pol["s_pm10Lag"] = df_graz_pol["s_pm10"].shift(1) 
    df_graz_pol["w_pm10Lag"] = df_graz_pol["w_pm10"].shift(1) 

# Drop data from 26th to 30th March (Sahara Dust)

In [ ]:
print("Sahara dust values (pollutants)")
display(df_graz_pol[(df_graz_pol.index >= "2020-03-26") & (df_graz_pol.index <= "2020-03-30")])
print("Sahara dust values (meteorogical)")
display(df_graz_met[(df_graz_met.index >= "2020-03-26") & (df_graz_met.index <= "2020-03-30")])


index_to_drop_graz = df_graz_pol[(df_graz_pol.index >= "2020-03-26") & (df_graz_pol.index <= "2020-03-30")].index

if drop_graz_sahara_measurements:
    print("******************************\nDrop Sahara measurements\n******************************")
    df_graz_pol.drop(index_to_drop_graz,inplace=True)
    df_graz_met.drop(index_to_drop_graz,inplace=True)

# Drop data annually 1st to 3rd January (New year)

In [ ]:

li_drop_dates_new_year = ['01-01', '01-02','01-03']

print("New year values (pollutants)")
display(df_graz_pol[df_graz_pol.index.strftime('%m-%d').isin(li_drop_dates_new_year)])

print("New year values (meteorological)")
display(df_graz_met[df_graz_met.index.strftime('%m-%d').isin(li_drop_dates_new_year)])

index_to_drop_graz = df_graz_pol[df_graz_pol.index.strftime('%m-%d').isin(li_drop_dates_new_year)].index

if drop_graz_newYear_measurements:
    print("******************************\nDrop NewYear measurements\n******************************")
    df_graz_met.drop(index_to_drop_graz,inplace=True)
    df_graz_pol.drop(index_to_drop_graz,inplace=True)
   

# Conclusion:
### If we do not use feature e_windDirDeg,e_peakwindsp,  e_peakwindsp e_windDirClass we just have a total number of NaN values of: 303 NaNs
### Those values cannot be imputed since they are location dependend

# Create seperate models (per station)

In [153]:


dict_df_stations = {'d':pd.DataFrame(),'w':pd.DataFrame(),'s':pd.DataFrame(),'n':pd.DataFrame(),'e':pd.DataFrame()}


for key,value in dict_df_stations.items():
    li_met_graz_col_names = [x for x in df_graz_met.keys() if x.startswith(key)] # get all column names for a station
    li_pol_graz_col_names = [x for x in df_graz_pol.keys() if x.startswith(key)] # get all column names for a station
    dict_df_stations[key] = pd.concat([df_graz_met[li_met_graz_col_names],df_graz_pol[li_pol_graz_col_names]],axis=1)


In [154]:
def rename_columns(df)->pd.DataFrame:
    li_col_names = df.columns
    li_new_names = [elem.split('_')[1] if len(elem.split('_'))>1 else elem for elem in li_col_names]
    
    dict_rename = dict(zip(li_col_names,li_new_names))
    return  df.rename(columns=dict_rename)

In [155]:
for key in dict_df_stations:
    dict_df_stations[key] = rename_columns(dict_df_stations[key])

# Drop data annually 1st to 3rd January (New year)

In [ ]:
li_drop_dates_new_year = ['01-01', '01-02','01-03']

for df in dict_df_stations:  

    print("New year values: "+df)
    display(dict_df_stations[df][dict_df_stations[df].index.strftime('%m-%d').isin(li_drop_dates_new_year)])
    index_to_drop_graz = dict_df_stations[df][dict_df_stations[df].index.strftime('%m-%d').isin(li_drop_dates_new_year)].index

    if drop_graz_newYear_measurements:
        print("******************************\nDrop NewYear measurements\n******************************")
        dict_df_stations[df].drop(index_to_drop_graz,inplace=True)

# Add Features:
 - day of year
 - isholiday
 - dayBeforH
 - dayAfterH

In [162]:

for df in dict_df_stations:
    if dayOfYearSinusFunction:  
        dict_df_stations[df]["dayOfYear"] = [
        np.sin(2 * np.pi * get_day_of_year_adjusted(date)[0] / get_day_of_year_adjusted(date)[1])
        for date in dict_df_stations[df].index.to_pydatetime()
                                           ]       
    else:
    # no sin function 
        dict_df_stations[df]["dayOfYear"] =  [get_day_of_year(date) for date in dict_df_stations[df].index.to_pydatetime()]
    
    dict_df_stations[df]["holiday"] =  [check_holiday(date,at_holidays) for date in dict_df_stations[df].index]
    dict_df_stations[df]["dayAfterHoliday"] =  [check_holiday(date-datetime.timedelta(days=1),at_holidays) for date in dict_df_stations[df].index]
    dict_df_stations[df]["dayBeforeHoliday"] =  [check_holiday(date+datetime.timedelta(days=1),at_holidays) for date in dict_df_stations[df].index]
    dict_df_stations[df]["weekend"] =  [check_weekend(date) for date in dict_df_stations[df].index]


# Save all stations seperately

In [163]:
for name,df in dict_df_stations.items():
    filepath = Path('../datasets/data_per_station/'+name+'.csv')
    dict_df_stations[name].to_csv(filepath,sep=',')

# Save pre-processed data from Graz

In [164]:

df_graz_met["dayOfYear"] =  [get_day_of_year(date) for date in df_graz_met.index.to_pydatetime()]
df_graz_met["g_holiday"] =  [check_holiday(date,at_holidays) for date in df_graz_met.index]
df_graz_met["g_dayAfterHoliday"] =  [check_holiday(date-datetime.timedelta(days=1),at_holidays) for date in df_graz_met.index]
df_graz_met["g_dayBeforeHoliday"] =  [check_holiday(date+datetime.timedelta(days=1),at_holidays) for date in df_graz_met.index]
df_graz_met["weekend"] =  [check_weekend(date) for date in df_graz_met.index]

filepath_graz_met_preproc = Path('../datasets/data_graz_meteorology_preprocessed.csv')
filepath_graz_pol_preproc = Path('../datasets/data_graz_air_pollutants_preprocessed.csv')

df_graz_pol.to_csv(filepath_graz_pol_preproc,sep=',') 
df_graz_met.to_csv(filepath_graz_met_preproc,sep=',') 

In [ ]:
date = datetime.date(2020,12,31)
print(check_holiday(date+datetime.timedelta(days=1),at_holidays))
print(check_holiday(date-datetime.timedelta(days=1),at_holidays))
print(check_holiday(date,at_holidays))